# Test connection to ORACLE Database using Python

## Helpful documentation
- [Introduction to the Python Driver for Oracle Database](https://python-oracledb.readthedocs.io/en/latest/user_guide/introduction.html#introduction-to-the-python-driver-for-oracle-database)
- [Python python-oracledb Driver](https://oracle.github.io/python-oracledb/)
- [Python: Read Data from Oracle Database](https://kontext.tech/article/1019/python-read-data-from-oracle-database)

## Connecting to the database via the ORACLE "thin" client and selecting one record with its output via "print"
- It is expected that only one record is selected. [5.1.1. Fetch Methods](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#fetch-methods)
- Query parameter binding is done through **variable bindings**
- Password entry is done securely via the python package: [getpass - Portable password input](https://pypi.org/project/getpass4/)

In [1]:
    import oracledb
    import getpass
    
    username = "CUSTDOC"
    userpwd = getpass.getpass("Enter password: ")

    host="localhost"
    port="1521"
    service_name="XEPDB1"
    dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
   

Enter password:  ········


In [2]:
import oracledb
import os
try:
    connection = oracledb.connect(dsn=dsn)
    print("✅ Successfully connected to XEPDB1!")
    
    # Приклад виконання запиту
    cursor = connection.cursor()
    cursor.execute("SELECT SYSDATE FROM DUAL")
    (current_date,) = cursor.fetchone()
    print(f"The current date in the database: {current_date}")
except oracledb.Error as e:
    print(f"❌ Connection error: {e}")

finally:
   if connection:
        connection.close()
        print("✅ Connection closed")


✅ Successfully connected to XEPDB1!
The current date in the database: 2025-11-05 06:20:42
✅ Connection closed


## Trying to read data from the directory from the table ```CUSTDOC.CUST$D$DOCTYPE```

- Example of an SQL - query

```sql
SELECT A.TYPEDOC, A.NAMETYPE FROM CUSTDOC.CUST$D$DOCTYPE A;
```
**Data metadata definition:** [5.1.3. Query Column Metadata](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#query-column-metadata) and data type mapping: [5.1.4. Fetch Data Types](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#fetch-data-types)

In [5]:
sql_query = """
    SELECT A.TYPEDOC, A.NAMETYPE
    FROM CUSTDOC.CUST$D$DOCTYPE A
"""
connection = None
cursor = None

try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    
    print("-" * 50)
    print(f"{column_names[0]:<15} {column_names[1]}")
    print("-" * 50)
    rows = cursor.fetchall()
    if rows:
        for row in rows:
            print(f"{row[0]:<15} {row[1]}")
    else:
     print("The query did not return any rows.")
     print("-" * 50)
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Oracle error: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")


--------------------------------------------------
TYPEDOC         NAMETYPE
--------------------------------------------------
NAT_ID          National Identity Document
FOR_PASS        International Passport
BIRTH_CERT      Birth Certificate	
TAX_ID          Tax Identification Number
DRV_LIC         	Drivers License
COI             Certificate of Incorporation
REG_EXT         Company Register Extract
PoA             Power of Attorney
AoA             Articles of Association ( Charter )

The cursor is closed.
The connection to the database has been closed.


## We are trying to read the data in such a way that the resulting data set can be transformed into a Dictionary.
In this example, once you have a dictionary, it's easy to load it into pandas or perform other processing using Python tools.

In [6]:
import oracledb

sql = """SELECT A.TYPEDOC, A.NAMETYPE
    FROM CUSTDOC.CUST$D$DOCTYPE A
    """
try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql)
    columns = [col[0] for col in cursor.description]
    cursor.rowfactory = lambda *args: dict(zip(columns, args))
    data = cursor.fetchall()
    print(data)

except oracledb.Error as e:
    error_obj, = e.args
    print(f"Oracle error: {error_obj.code} - {error_obj.message}")
    
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")


[{'TYPEDOC': 'NAT_ID', 'NAMETYPE': 'National Identity Document'}, {'TYPEDOC': 'FOR_PASS', 'NAMETYPE': 'International Passport'}, {'TYPEDOC': 'BIRTH_CERT', 'NAMETYPE': 'Birth Certificate\t'}, {'TYPEDOC': 'TAX_ID', 'NAMETYPE': 'Tax Identification Number'}, {'TYPEDOC': 'DRV_LIC', 'NAMETYPE': '\tDrivers License'}, {'TYPEDOC': 'COI', 'NAMETYPE': 'Certificate of Incorporation'}, {'TYPEDOC': 'REG_EXT', 'NAMETYPE': 'Company Register Extract'}, {'TYPEDOC': 'PoA', 'NAMETYPE': 'Power of Attorney'}, {'TYPEDOC': 'AoA', 'NAMETYPE': 'Articles of Association ( Charter )'}]

The cursor is closed.
The connection to the database has been closed.


## Inserting data into the database
- [5.2. INSERT and UPDATE Statements](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#insert-and-update-statements)
  [9. Managing Transactions](https://python-oracledb.readthedocs.io/en/latest/user_guide/txn_management.html#managing-transactions)

In [9]:
import oracledb
import json
import os
file_path = 'artists.json'
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        artists_data = json.load(f)
        
    print("JSON file successfully read into Python list.")
    print(f"The first element: {artists_data['rows'][0]['artist_en']}")
    connection = oracledb.connect(dsn)
    cursor = connection.cursor()
    sql = """insert into CUSTDOC.CUST$ARTIST 
              ( artist_en, artist_ua, photo_url, work_url, work_title_en, work_title_ua ) VALUES 
              ( :a_artist_en, :a_artist_ua, :a_photo_url, :a_work_url, :a_work_title_en, :a_work_title_ua  )"""
    for artist in artists_data['rows']:
        print( f" INSERT DATA ROW:  {artist['artist_en']}")
        cursor.execute(sql, 
                       {
                        "a_artist_en": artist['artist_en'], 
                        "a_artist_ua": artist['artist_ua'], 
                        "a_photo_url": artist['photo_url'], 
                        "a_work_url":  artist['work_url'], 
                        "a_work_title_en": artist['work_title_en'], 
                        "a_work_title_ua": artist['work_title_ua']
                       }
         )
    connection.commit()
except FileNotFoundError:
    print(f"Error: File not found at path{file_path}")
except json.JSONDecodeError:
    print("Error: Invalid JSON format in file.")
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Oracle error: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")

JSON file successfully read into Python list.
The first element: Pablo Picasso
 INSERT DATA ROW:  Pablo Picasso
 INSERT DATA ROW:  Salvador Dalí
 INSERT DATA ROW:  Francisco Goya
 INSERT DATA ROW:  Diego Velázquez
 INSERT DATA ROW:  Joan Miró

The cursor is closed.
The connection to the database has been closed.


## Trying to read the inserted data

In [10]:
sql_query = """
    SELECT A.artist_en, A.artist_ua 
    FROM CUSTDOC.CUST$ARTIST  A
"""
connection = None
cursor = None

try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    
    print("-" * 50)
    print(f"{column_names[0]:<15} {column_names[1]}")
    print("-" * 50)
    rows = cursor.fetchall()
    if rows:
        for row in rows:
            # Виведення даних з форматуванням
            # :<15 означає вирівнювання по лівому краю, займаючи 15 символів
            print(f"{row[0]:<15} {row[1]}")
    else:
        print("The query did not return any rows.")
    print("-" * 50)
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Oracle error: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")

--------------------------------------------------
ARTIST_EN       ARTIST_UA
--------------------------------------------------
Pablo Picasso   Пабло Пікассо
Salvador Dalí   Сальвадор Далі
Francisco Goya  Франсіско Гойя
Diego Velázquez Дієго Веласкес
Joan Miró       Жоан Міро
--------------------------------------------------

The cursor is closed.
The connection to the database has been closed.
